## Trabalho final desenvolvido pelos alunos:
- Itor Carlos Souza Queiroz
- Lanna Luara Novaes Silva

Instalação das dependências necessárias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, color, filters, feature
from skimage.util import img_as_float
from skimage.filters import sobel
from skimage.io import imread
from roboflow import Roboflow
import roboflow
import zipfile
import os
import requests



Define o diretório onde ficará as imagens do dataset que está sendo usado no trabalho

In [ ]:
caminho_diretorio = 'dices/'
arquivos = [f for f in os.listdir(caminho_diretorio) if f.endswith(('.png', '.jpg', '.jpeg'))]

Cria uma função para ler as imagens da pasta "dices" para exibir as imagens antes de realizar as próximas operações

In [ ]:
def exibe_grade(lista_imagens, titulos, mapa_cor=None):
    n_imagens = len(lista_imagens)
    numero_colunas = 4
    numero_linhas = (n_imagens // numero_colunas) + (1 if n_imagens % numero_colunas != 0 else 0)

    plt.figure(figsize=(15, numero_linhas * 4))
    for i, imagem in enumerate(lista_imagens):
        plt.subplot(numero_linhas, numero_colunas, i + 1)
        plt.imshow(imagem, cmap=mapa_cor if len(imagem.shape) == 2 else None)
        plt.axis("off")
        plt.title(titulos[i])
    plt.tight_layout()
    plt.show()

Definição de uma função para converter as imagens para a escala de cinza. Primeiramente será removido o canal alpha, referente a transparência, caso o mesmo exista.

In [ ]:
def converter_para_cinza(lista_entrada):
    lista_saida = []
    for img in lista_entrada:
        # Remove canal alpha (transparência) se existir
        if len(img.shape) == 3 and img.shape[-1] == 4:
            img = img[:, :, :3]

        cinza = color.rgb2gray(img)
        lista_saida.append(cinza)
    return lista_saida

Exibição das imagens originais, ou seja, anterior as operações do trabalho

In [ ]:
imagens_originais = []
for nome in arquivos:
    img = io.imread(os.path.join(caminho_diretorio, nome))
    imagens_originais.append(img)


print("Exibindo imagens originais...")
exibe_grade(imagens_originais, arquivos)

Aplicação da operação de conversão para a escala de cinza e a exibição das imagens do mesmo

In [ ]:
imagens_cinza = converter_para_cinza(imagens_originais)

print("Exibindo imagens em escala de cinza...")
# Importante: para imagens em cinza, passamos cmap='gray'
exibe_grade(imagens_cinza, arquivos, mapa_cor='gray')

Aplicação da operação de suavização em cada imagem do dataset e a exibição das mesmas após a operação.

In [ ]:
# Aplicando a suavização com sigma=1.5 em todas as imagens
# O sigma define a "força" do borrão. 1.5 é um valor equilibrado para OCR.
imagens_suavizadas = [filters.gaussian(img, sigma=1.5) for img in imagens_cinza]

print(f"{len(imagens_suavizadas)} imagens foram suavizadas com sucesso!")

# Exibindo o resultado usando a função que criamos anteriormente
print("Exibindo imagens suavizadas (Noise Reduction):")
exibe_grade(imagens_suavizadas, arquivos, mapa_cor='gray')

Operação de detectação de bordas dos objetos de cada imagem presente no dataset escolhido.

In [ ]:
imagens_limites = [feature.canny(img, sigma=2) for img in imagens_suavizadas]

print(f"Bordas extraídas de {len(imagens_limites)} imagens.")

# Exibindo o resultado
# O Canny retorna uma imagem booleana (True para borda, False para fundo)
# O mapa 'gray' mostrará as bordas em branco sobre fundo preto
print("Exibindo detecção de bordas (Canny Edge Detection):")
exibe_grade(imagens_limites, arquivos, mapa_cor='gray')